In [1]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)
from google.genai import types
from google.adk.tools import google_search
from google.genai import types
import asyncio
import pprint
import sys

from google.adk.tools.agent_tool import AgentTool

In [2]:
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
model_name = "gemini-2.0-flash"

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

short_config = types.GenerationConfig(
    max_output_tokens=500, # Limit response to 500 tokens
)

model_config = Gemini(
    model=model_name,
    retry_options=retry_config,
    generation_config=short_config,
    api_key=GOOGLE_API_KEY,
)

In [3]:
instructions = """
    You are a helpful shopping assistant. Use the tools provided to find products, compare prices, 
    and provide recommendations based on user preferences.
    When using the google_search tool, be specific with your queries to get the best results.
    Focus on finding high-quality products and good deals.
    Return up to 3 product recommendations with brief descriptions and prices in following JSON format:
    [
        {
            "seller": str = "Seller Name",
            "brand": str = "Brand Name",
            "product": str = "Product Name",
            "price": float = "Product Price",
            "user_review_score": float = "User Review Score"
        },
        ...
    ]
"""

shopping_agent = Agent(
    model=model_config,
    name="shopping_agent",
    description="An agent for shopping assistance",
    tools=[google_search],
    output_key="shopping_recommendations",
    instruction=instructions,
)
print("✅ shopping_agent created.")

✅ shopping_agent created.


In [4]:
instructions = """
    You are a sustainability evaluation agent. Use the tools provided to research companies,
    focusing on their environmental impact, ethical practices, and sustainability initiatives.
    You should each brand provided in the list {shopping_recommendations} and evaluate their sustainability
    based on recent data. You need to use the google_search tool to find relevant information.
    Your key evaluation results should include:
    - Environmental Impact: Assess the brand's carbon footprint, resource usage, and waste management.
    - Ethical Practices: Evaluate labor practices, fair trade certifications, and community engagement.
    - Animal Welfare: Consider policies on animal testing and cruelty-free certifications.
    - Sustainability Initiatives: Look for programs or policies aimed at reducing environmental impact.
    For each of those categories make a score between 1 and 5, where 1 is poor and 5 is excellent.
    Scorring rules:
    - 1: Very poor performance, significant negative impact or unethical practices.
    - 2: Below average performance, some negative impact or questionable practices.
    - 3: Average performance, mention of ethical practices on company website but with no external validation.
    - 4: Good performance, mention of ethical practices on company website with some external validation.
    - 5: Excellent performance, mention of ethical practices on company website with strong external validation.
    Combine these scores to provide an overall sustainability rating for each brand.
    For each brand, return a brief evaluation in the following JSON format:
    [
        {
            "brand": str = "Brand Name",
            "environment_impact_score": int = 1-5,
            "ethical_practices_score": int = 1-5,
            "animal_welfare_score": int = 1-5,
            "sustainability_initiatives_score": int = 1-5,
            "overall_sustainability_rating": float = 1-5,
            "summary": str = "Brief summary of the evaluation for each of the categories and
                justification for the overall rating",
        },
        ...
    ]
"""

sustainability_evaluation_agent = Agent(
    model=model_config,
    name="sustainability_evaluation_agent",
    description="An agent for sustainability evaluation",
    tools=[google_search],
    output_key="sustainability_evaluation",
    instruction=instructions,
)
print("✅ sustainability_evaluation_agent created.")

✅ sustainability_evaluation_agent created.


In [5]:
instructions = """
    You are a sustainable shopping assistant. Your task is to combine the shopping recommendations
    from the shopping_agent with the sustainability evaluations from the sustainability_evaluation_agent.
    Provide the user with a final list of product recommendations that balance quality, price, and sustainability.
    For each product recommendation, include the sustainability evaluation of the brand.
    Make single final recommendation for the best product in the following JSON format:
    {
            "brand": str = "Brand Name",
            "environment_impact_score": int = 1-5,
            "ethical_practices_score": int = 1-5,
            "animal_welfare_score": int = 1-5,
            "sustainability_initiatives_score": int = 1-5,
            "overall_sustainability_rating": float = 1-5,
            "user_review_score": float = "User Review Score"
            "summary": str = "Brief summary of the evaluation for each of the categories and
                justification for the overall rating",
    }
    Sustainability evaluation should be prioritized when making final recommendations.
"""

root_agent = Agent(
    model=model_config,
    name="sustainable_shopping_assistant",
    description="An agent that assists users in making sustainable shopping choices",
    tools=[AgentTool(agent=shopping_agent), AgentTool(agent=sustainability_evaluation_agent)],
    instruction=instructions,
)

In [6]:
runner = InMemoryRunner(
    agent=root_agent,
    plugins=[
        LoggingPlugin()
    ]
)

In [7]:
old_stdout = sys.stdout

log_file = open("agent_logs.log","w", encoding="utf-8")
sys.stdout = log_file

response = await runner.run_debug(
    "Find chrismas chocolate gift boxes."
)

In [8]:
sys.stdout = old_stdout
log_file.close()

In [9]:
# Print all responses
for i, event in enumerate(response):
    content = event.content
    for part in content.parts:
        if part.function_call:
            function_call = part.function_call
            print(f"*** Function Call in Agent {function_call.name} ***")
            pprint.pp(function_call.args["request"], depth=1)
        if part.function_response:
            function_response = part.function_response
            print(f"*** Function Response in Agent {function_response.name} ***")
            pprint.pp(function_response.response["result"], depth=1)
    metadata = event.usage_metadata
    if metadata:
        tokens = metadata.total_token_count
        print(f"Total tokens used in this event: {tokens}")


*** Function Call in Agent shopping_agent ***
'chrismas chocolate gift boxes'
Total tokens used in this event: 318
*** Function Response in Agent shopping_agent ***
('Here are some Christmas chocolate gift box options for 2025:\n'
 '\n'
 '*   **Compartes:** Offers a variety of Christmas chocolate gifts, including '
 'the Green Pine Christmas Chocolate Gift Box. They also have advent '
 'calendars, stocking stuffers, and chocolate bark.\n'
 '*   **Cacao and Cardamom:** Offer luxury Christmas chocolate gift boxes with '
 'selections like the Holiday 16-Piece Box, the Jewelry Box (24 Pieces), and '
 'the Candy Cane Confections Gift Box. They also have a Holiday Chocolate '
 'Advent Calendar.\n'
 '*   **Mueller Chocolate Co.:** They offer several Christmas chocolate gift '
 'boxes, including the Crowd Pleaser Gift Set, Classic Chocolate Gift Tower, '
 'Seasonal Dark/Milk Chocolate Nonpareils, and Dark/Milk Chocolate Sea Salt '
 'Caramel Piece Box.\n'
 '*   **Hotel Chocolat:** Offers a rang

In [10]:
# Final recommendations:
print("*** Final Recommendations ***")
pprint.pp(response[-1].content.parts[-1].text, depth=1)

*** Final Recommendations ***
('```json\n'
 '{\n'
 '    "brand": "Lindt",\n'
 '    "environment_impact_score": 4,\n'
 '    "ethical_practices_score": 4,\n'
 '    "animal_welfare_score": 5,\n'
 '    "sustainability_initiatives_score": 4,\n'
 '    "overall_sustainability_rating": 4.25,\n'
 '    "user_review_score": 4.5,\n'
 '    "summary": "Lindt demonstrates a strong commitment to sustainability. '
 'Their environmental efforts include reducing deforestation, achieving '
 'net-zero emissions by 2050, and designing recyclable packaging. Ethically, '
 'Lindt focuses on improving the livelihoods of cocoa farmers through the '
 'Lindt & Sprüngli Farming Program. They also have a policy banning animal '
 "testing. Lindt's comprehensive approach and clear targets indicate a strong "
 'dedication to sustainability."\n'
 '}\n'
 '```')
